#  THE ION–ION POTENTIAL:
The two-body (Na and Cl) potential consists of the sum of the attractive or repulsive point charge Coulomb energy and the repulsive Pauli energy. The potential energy function \( U(r_{ij}) \) models the interaction between particles (Na and Cl ions) based on their separation distance \( r_{ij} \). The function has different forms depending on the sign of the interacting particles, denoted as (\(+ -\)) or (\(++\) or \(--\)).


$$
U\left(r_{i j}\right)=\left\{\begin{array}{l}{-\frac{e^{2}}{4 \pi \epsilon_{0} r_{i j}}+\alpha e^{-r_{i j} / \rho}+b\left(\frac{c}{r_{i j}}\right)^{12}} (\mathrm{+ -})\\ {+\frac{e^{2}}{4 \pi \epsilon_{0} r_{i j}}+b\left(\frac{c}{r_{i j}}\right)^{12}} (\mathrm{++ or --}) \end{array}\right.
$$

1. **Coulombic Term**: Describes the electrostatic interaction between charged particles. For (\(+ -\)), it's attractive, and for (\(++\) or \(--\)), it's repulsive.

   \[ -\frac{e^{2}}{4\pi\epsilon_{0} r_{ij}} \]

2. **Pauli Exclusion Term**: Introduces a repulsive term based on the Pauli exclusion principle.

   \[ \alpha e^{-r_{ij} / \rho} \]

3. **Stabilizing Term**: Introduced to stabilize the computation at small distances.

   \[ b\left(\frac{c}{r_{ij}}\right)^{12} \]

4. **Constant Terms**: The overall Coulomb force constant \( ke^2 \), Pauli exclusion constant \( \alpha \), and the stabilization constants \( b \) and \( c \) are given.

   \[ ke^2 = \frac{e^{2}}{4\pi\epsilon_{0}} = 1.44\; \mathrm{eV-nm} \]
   \[ \alpha = 1.09e3\; \mathrm{eV} \]
   \[ b = 1.0\; \mathrm{eV} \]
   \[ c = 0.01\; \mathrm{nm} \]

